In [1]:
import pdfplumber
import re

The following code block will scrape all text from the parcel report PDF -- which was generated from scraping KGIS.

In [2]:
pdf = pdfplumber.open("Parcel_Report.pdf")
all_pages = pdf.pages
cleaned_lines = []
cleaned_list_of_dicts = []
for each_page in all_pages:
    text_of_each_page = each_page.extract_text()
    all_lines = text_of_each_page.splitlines()
    for each_line in all_lines:
        if each_line == "Address":
            continue
        elif each_line == "Owner Card  Mailing Label  Map and Details Report":
            continue
        elif each_line == "KGIS Parcel Report":
            continue
        else:
            cleaned_lines.append(each_line)
each_dict = {}
for index, cleaned_line in enumerate(cleaned_lines):
    if re.match(r"^09[0-9A-Z]{5,}$", cleaned_line):
        # parcel_id
        each_dict["parcel_id"] = cleaned_line
        each_dict["name"] = cleaned_lines[index + 1]
        cleaned_list_of_dicts.append(each_dict)
        each_dict = {}
    else:
        continue

In [3]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager

/Users/areena.arora/.pyenv/versions/3.10.0/lib/python3.10/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [35]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 105.0.5195
Get LATEST chromedriver version for 105.0.5195 google-chrome
Driver [/Users/areena.arora/.wdm/drivers/chromedriver/mac64/105.0.5195.52/chromedriver] found in cache
/var/folders/81/2s24wkdd02j9528vnjsx7m080000gn/T/ipykernel_3789/1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [36]:
def get_all_data(parcel_number, agree):
    driver.get('https://propertyinfo.knoxcountytn.gov/search/commonsearch.aspx?mode=realprop')
    if agree:
        driver.find_element(By.ID, "btAgree").click()
    driver.find_element(By.ID, "inpParid").send_keys(parcel_number)
    driver.find_element(By.ID, "btSearch").click()
    error_passing = driver.find_elements(By.ID, "searchResults")
    # print(error_passing)
    if len(error_passing) > 0:
        error_passing[0].find_element(By.CLASS_NAME, "SearchResults").click()
    driver.find_element(By.XPATH, '//*[@id="sidemenu"]/ul/li[3]/a').click()
    parcel_id_dirty = driver.find_element(By.CLASS_NAME, "DataletHeaderTop").text
    parcel_id = parcel_id_dirty.split(": ")[1]
    sales_data = []
    try:
        sales_details = driver.find_element(By.ID, "Sales Summary")
        rows = sales_details.find_elements(By.TAG_NAME, "tr")[1:]
        for row in rows:
            all_data = row.find_elements(By.TAG_NAME, "td")
            if len(all_data) < 5:
                print("insufficient data")
                continue
            else:
                data = {}
                date = all_data[0].text
                book = all_data[1].text
                page = all_data[2].text
                price = all_data[3].text
                grantee = all_data[-1].text
                data["date"] = date
                data["book_number"] = book
                data["page"] = page
                data["sale_price"] = price
                data["grantee_name"] = grantee
                data["parcel_id_new"] = parcel_id
                sales_data.append(data)
    except:
        data = {}
        data["date"] = "Not found"
        data["book_number"] = "Not found"
        data["page"] = "Not found"
        data["sale_price"] = "Not found"
        data["grantee_name"] = "Not found"
        data["parcel_id_new"] = "Not found"
        sales_data.append(data)
    return(sales_data)

In [6]:
# get_all_data("094LF015", True)
all_sales_data = []

In [37]:
agree = True
for index, each_parcel_dict in enumerate(cleaned_list_of_dicts[900:999]):
    sales_data = get_all_data(each_parcel_dict["parcel_id"], agree)
    agree = False
    all_sales_data.extend(sales_data)
    print(index, "done")

insufficient data
0 done
insufficient data
1 done
insufficient data
2 done
insufficient data
3 done
insufficient data
4 done
insufficient data
5 done
insufficient data
6 done
insufficient data
7 done
insufficient data
8 done
insufficient data
9 done
insufficient data
10 done
insufficient data
11 done
insufficient data
12 done
insufficient data
13 done
insufficient data
14 done
insufficient data
15 done
insufficient data
16 done
insufficient data
17 done
insufficient data
18 done
insufficient data
19 done
insufficient data
20 done
insufficient data
21 done
insufficient data
22 done
insufficient data
23 done
insufficient data
24 done
insufficient data
25 done
insufficient data
26 done
insufficient data
27 done
insufficient data
28 done
insufficient data
29 done
insufficient data
30 done
31 done
insufficient data
32 done
insufficient data
33 done
insufficient data
34 done
insufficient data
35 done
insufficient data
36 done
37 done
insufficient data
38 done
insufficient data
39 done
insuff

In [38]:
pd.set_option('display.max_columns', None)
df = pd.DataFrame(all_sales_data)

In [39]:
df

,date,book_number,page,sale_price,grantee_name,parcel_id_new
0,08/16/1990,00002017,0000594,$0,PARKEY SYLVIA ROCHELLE,095GP014
1,07/14/1989,00001982,0001091,"$32,000",PARKEY SYLVIA ROCHELLE,095GP014
2,11/30/1994,00002160,0001032,"$20,686",LEE MARY N,095GN008
3,06/26/1973,00001521,0000692,$0,KNOXVILLES COMMUNITY DEV CORP,095GN008
4,10/18/1996,00002229,0000419,$0,BROWN MARTHA S COBB & JERVIS S,095GP010
...,...,...,...,...,...,...
3554,12/05/2014,20141210,0031895,$0,SMITH PATRICK A,095AM016
3555,08/15/2014,20140827,0012176,"$350,000",SMITH PATRICK A,095AM016
3556,07/06/2001,20010709,0001644,"$310,000",STEIN REAL ESTATE LIMITED PARTNERSHIP,095AM016
3557,05/14/1993,00002106,0000454,"$200,000",GILBERT DAVID,095AM016


In [40]:
df.to_csv("all_scraped.csv")